# Utilities Real Estate Analysis

The goal of this analysis is to see which stocks in real estate industry had the highest returns.

In [4]:
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import requests

### Getting Utilities Industry Tickers

In [5]:
# URL of the Yahoo Finance utilities sector screener
url = "https://finance.yahoo.com/screener/predefined/ms_real_estate/"

# Send an HTTP GET request to the URL
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Use pandas to read the HTML table directly from the response
    dfs = pd.read_html(response.text)

    # The desired table is the first one in the list
    real_estate = dfs[0]

    # Display the scraped data (optional)
    print(real_estate)

else:
    print(f"Failed to retrieve data. Status code: {response.status_code}")

    Symbol                                            Name  Price (Intraday)  \
0      PLD                                  Prologis, Inc.            123.81   
1      AMT                      American Tower Corporation            185.75   
2     EQIX                                   Equinix, Inc.            759.42   
3      PSA                                  Public Storage            286.98   
4   SPG-PJ                      Simon Property Group, Inc.             57.60   
5      CCI                               Crown Castle Inc.            104.93   
6      SPG                      Simon Property Group, Inc.            118.37   
7     WELL                                  Welltower Inc.             83.79   
8   PSA-PH                                  Public Storage             25.13   
9        O                       Realty Income Corporation             58.84   
10  PSA-PK                                  Public Storage             21.45   
11     DLR                      Digital 

### Visualising the Data

In [8]:
# Define the benchmark symbols and corresponding line styles
benchmark_symbols = {'IWDP.MI': 'dash'}


# Function to fetch and analyze stock data
def analyze_stock_data(symbol):
    real_estate_stock = yf.download(symbol, start='2017-01-01', end='2023-01-01')
    if real_estate_stock.empty:
        print(f"No data available for symbol: {symbol}")
        return None
    real_estate_stock_returns = real_estate_stock['Adj Close'].pct_change().dropna()
    real_estate_stock_cumulative_returns = (1 + real_estate_stock_returns).cumprod()
    return real_estate_stock_cumulative_returns

# Create an empty dictionary to store cumulative returns for utility companies and benchmark indices
returns_dict = {}

# Loop through the 'utilities' dataframe to analyze each company
for symbol in real_estate['Symbol']:
    cumulative_returns = analyze_stock_data(symbol)
    if cumulative_returns is not None:
        returns_dict[symbol] = cumulative_returns

# Fetch data for the benchmark indices
for symbol, line_style in benchmark_symbols.items():
    benchmark_stock = yf.download(symbol, start='2017-01-01', end='2023-01-01')
    if benchmark_stock.empty:
        print(f"No data available for benchmark symbol: {symbol}")
        continue
    benchmark_stock_returns = benchmark_stock['Adj Close'].pct_change().dropna()
    benchmark_stock_cumulative_returns = (1 + benchmark_stock_returns).cumprod()
    returns_dict[symbol] = benchmark_stock_cumulative_returns

# Check if any data is available in the returns_dict
if not returns_dict:
    print("No data available for utility companies or benchmark indices.")
else:
    # Create interactive plot using plotly
    fig = go.Figure()
    for symbol, returns in returns_dict.items():
        line_style = 'dash' if symbol in benchmark_symbols else 'solid'
        fig.add_trace(go.Scatter(x=returns.index, y=returns, mode='lines', name=symbol, line=dict(dash=line_style)))

    # Update layout for better interactivity
    fig.update_layout(
        title='Real Estate Industry vs. Benchmark Indices',
        xaxis_title='Date',
        yaxis_title='Cumulative Returns',
        xaxis_rangeslider_visible=True,
        showlegend=True
    )

    fig.show()


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

### Best Performing Stocks

In [9]:
# Convert the sample data to a DataFrame
df = pd.DataFrame(returns_dict)
# Sort the DataFrame by the last row (latest date) in descending order
sorted_df = df.iloc[-1].sort_values(ascending=False)

# Select the top 5 best-performing tickers
top_5_best_performing = sorted_df.head(5)

# Display the table
print(top_5_best_performing)

CSGP    4.151491
SBAC    2.725764
PLD     2.498507
CBRE    2.437757
EXR     2.347143
Name: 2022-12-30 00:00:00, dtype: float64


### Summary

For the given period, the best performing stocks in the real estate industry were:

1. **CoStar Group Inc. (CSGP)** - Washington, DC-based provider of information, analytics, and marketing services to the commercial property industry in the United States, Canada, the United Kingdom, France, Germany, and Spain.

2. **SBA Communications Corp (SBAC)** - Real estate investment trust which owns and operates wireless infrastructure in the United States, Canada, Central America, South America, and South Africa.

3. **Prologis Inc. (PLD)** - Real estate investment trust headquartered in San Francisco, California that invests in logistics facilities. The company was formed through the merger of AMB Property Corporation and Prologis in June 2011, which made Prologis the largest industrial real estate company in the world.

4. **CBRE Group Inc. (CBRE)** - CBRE Group, Inc. is an American commercial real estate services and investment firm. The abbreviation CBRE stands for Coldwell Banker Richard Ellis. It is the world's largest commercial real estate services and investment firm.

5. **Extra Space Storage Inc. (EXR)** - Real estate investment trust headquartered in Salt Lake City, UT that invests in self storage facilities.